# Content 
This note contain three main points :
- EDA
- Univariate visualization
- Bivariate visualzation
- Multivariate visualization

And as we will find that, our data is imbalanced so we will try to treat this by 

-  imblearn 

then we do our clsaaification 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing libraries

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#import our train and test data 
train=pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')


In [ ]:
#display some rows from our train data 
train.head()

In [ ]:
#shape of our training  data
train.shape

In [ ]:
#some information about training data
train.info()

In [ ]:
# the number of nulls in training data
na_counts=train.isnull().sum()
na_counts

In [ ]:
#plotting the number of null values in each feature
plt.figure(figsize=[10,5]);
sns.barplot(na_counts.index.values,na_counts);
plt.xticks(rotation=90);

We note that there is many missing values in company_size and comany_type ,so if i dropped all the nulls in data , i will lose more than half my training dataset ,so i will drop these two columns

In [ ]:
#drop company_size , company_type
train.drop(['company_size','company_type'],axis=1,inplace=True)


after that we will find few rows with missing values so we will drop it to start analyze our clean data

In [ ]:
#dropping nulls in rest of our training data 
train.dropna(axis=0,inplace=True)
train.info()

In [ ]:
#check for duplicates
train.duplicated().sum()

# EDA

# Univariate visualization

Number of males and femals

In [ ]:
plt.figure(figsize=[15,7])
plt.subplot(1,2,1)
sorted_counts=train['gender'].value_counts()
plt.pie(sorted_counts,labels=sorted_counts.index,startangle=90,counterclock=False,wedgeprops={'width':0.4})
plt.axis('square');
plt.subplot(1,2,2)
sns.barplot(sorted_counts.index.values,sorted_counts)
#plt.yticks(range(np.arange(0,len(train)+1000,2000))
plt.yticks(range(0,14000,2000));

number of  relevant_exoeriance in each unique one

In [ ]:
#number of relevant experiance in each unique one
base_color=sns.color_palette()[2]
rects1=sns.countplot(data=train,x='relevent_experience',color=base_color);

 

Count of education level 

One alternative univariate plot type that you might see for categorical data is the waffle plot, also known as the square pie chart. While the standard pie chart uses a circle to represent the whole, a waffle plot is plotted onto a square divided into a 10x10 grid. Each small square in the grid represents one percent of the data, and a number of squares are colored by category to indicate total proportions. Compared to a pie chart, it is much easier to make precise assessments of relative frequencies.

In [ ]:
#creating waffle plot for education level
def percentage_blocks(df, var):
    """
    Take as input a dataframe and variable, and return a Pandas series with
    approximate percentage values for filling out a waffle plot.
    """
    # compute base quotas
    percentages = 100 * train[var].value_counts() / df.shape[0]
    counts = np.floor(percentages).astype(int) # integer part = minimum quota
    decimal = (percentages - counts).sort_values(ascending = False)

    # add in additional counts to reach 100
    rem = 100 - counts.sum()
    for cat in decimal.index[:rem]:
        counts[cat] += 1

    return counts


waffle_counts = percentage_blocks(train, 'education_level')

prev_count = 0
# for each category,
for cat in range(waffle_counts.shape[0]):
    # get the block indices
    blocks = np.arange(prev_count, prev_count + waffle_counts[cat])
    # and put a block at each index's location
    x = blocks % 10 # use mod operation to get ones digit
    y = blocks // 10 # use floor division to get tens digit
    plt.bar(x = x, height = 0.8, width = 0.8, bottom = y)
    prev_count += waffle_counts[cat]

    
# aesthetic wrangling
plt.legend(waffle_counts.index, bbox_to_anchor = (1, 0.5), loc = 6);
plt.axis('off');
plt.axis('square');


Let's see the distribution of training ours for all our candidates 

In [ ]:
#showing some statistic about training hours
ax = sns.boxplot(data=train['training_hours'], orient="v", palette="Set2")

In [ ]:
#showing the distribution of training hours
bin_edges = np.arange(0, train['training_hours'].max()+20, 20)
sns.distplot(train['training_hours'], bins = bin_edges, kde = False,
            hist_kws = {'alpha' : 1})


# Bivariate visualization

In [ ]:
train.head()

Number of each gender in each relevant experiance unique value

In [ ]:
#Number of each gender in each relevant experiance unique value
sns.set(style="whitegrid")
ax = sns.countplot(x="gender", hue="relevent_experience", data=train)
plt.title('Number of each gender in each relevant experiance unique value');

Let's see the number of each gender in each education level


In [ ]:
#Number of each gender in each education level
ct_counts = train.groupby(['gender', 'education_level']).size()

# Use Series.reset_index() to convert a series into a dataframe object
ct_counts = ct_counts.reset_index(name='count')
 
# Use DataFrame.pivot() to rearrange the data, to have gender type on rows
ct_counts = ct_counts.pivot(index = 'gender', columns = 'education_level', values = 'count')
 
 
sns.heatmap(ct_counts,annot=True,fmt='d');
plt.title('Number of each gender in each education level');

In [ ]:
# Convert the "gender" column from a plain object type into an ordered categorical type
gender_types = list(train['gender'].unique())
vclasses = pd.api.types.CategoricalDtype(ordered=True, categories=gender_types)
train['gender'] = train['gender'].astype(vclasses);

# Plot the Seaborn's FacetGrid
g = sns.FacetGrid(data = train, col = 'gender')
g.map(plt.hist, "training_hours");


In [ ]:
#statics for each gender with training hours 
ax=sns.boxplot(data=train,x='gender',y='training_hours')

In [ ]:
#number of each gender in each target unique
ax = sns.countplot(y="gender", hue="target", data=train)
plt.legend()

# Multivariate visualization

In [ ]:
#statistic for each gender in every education level & training_hours
ax=sns.boxplot(data=train,x='gender',y='training_hours',hue='education_level')

# Handling our data to make our classification

In [ ]:
#let's take alook again on our data set
train.head()

In [ ]:
train['gender']=train['gender'].replace({'Male':0,'Female':1,'Other':2})
train['enrolled_university']=train['enrolled_university'].replace({'no_enrollment':0,'Full time course':1,'Part time course':2})
train['relevent_experience']=train['relevent_experience'].replace({'No relevent experience':0,'Has relevent experience':1})
train['education_level']=train['education_level'].replace({'Graduate':1,'Masters':2,'Phd':3})

train['major_discipline']=train['major_discipline'].replace({'STEM':1, 'Humanities':2, 'Arts':3, 'Business Degree':4,'No Major':5,'Other':6})

train['experience']=train['experience'].replace({'>20':30,'<1':0})

train['last_new_job']=train['last_new_job'].replace({'>4':5,'1':1,'never':0,'4':4, '3':3, '2':2})


In [ ]:
train['experience']=train['experience'].replace({'>20':30,'<1':0,'15':15,'13':13,'7':7, '5':5, '16':16, '4':4, '11':11, '18':18, '19':19, '12':12,
       '10':10, '9':9, '2':2, '6':6, '14':14, '3':3, '8':8, '20':20, '17':17, '1':1})


In [ ]:
train['training_hours']=train['training_hours']/train['training_hours'].max()
train['experience']=train['experience']/train['experience'].max()

In [ ]:
train.head()

In [ ]:
#let's checking again for coulmns type to see the correlation
train.info()

Correlation factor between target and each column

In [ ]:
train.shape

In [ ]:
#correlation matrix
plt.figure(figsize=[10,10])
sns.heatmap(train.corr(),annot=True,fmt='.3f')
plt.title('Correlation factor between target and each column');

From correlation matrix we can define which features are strong related to our target and which not.

In [ ]:
#our features and target
X=train.drop(['enrollee_id','city','target'],axis=1) #these featured don't benefit us in our clsassification 
y=train['target']


# Treating our imbalance data 
As we see from our visualization , we have an imbalance data in our training data set , since the 0 in target is more higher than 1 

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter


rand=RandomOverSampler(random_state=42)
x_ros, y_ros = rand.fit_resample(X, y)
print(f"Imbalanced target class: {Counter(y)} Balanced target class:{Counter(y_ros)}")

In [ ]:
#splitting our data 
X_train, X_test, y_train, y_test = train_test_split(x_ros, y_ros, test_size=0.33, random_state=44, shuffle =True)

#Splitted Data
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

#  GradientBoostingClassifier

In [ ]:
#Import Libraries
from sklearn.ensemble import GradientBoostingClassifier
 

GBCModel = GradientBoostingClassifier(n_estimators=500,max_depth=50,random_state=33) 
GBCModel.fit(X_train, y_train)

#Calculating Details
print('GBCModel Train Score is : ' , GBCModel.score(X_train, y_train))
print('GBCModel Test Score is : ' , GBCModel.score(X_test, y_test))
 
#Calculating Prediction
y_pred = GBCModel.predict(X_test)
y_pred_prob = GBCModel.predict_proba(X_test)
 

As we see there is overfitting when we use GradientBoostingClassifier, so we will try to use random forest

# Random forest

In [ ]:
# applying  random forest
RandomForestClassifierModel = RandomForestClassifier(criterion = 'gini',n_estimators=500,max_depth=10,random_state=33) #criterion can be also : entropy 
RandomForestClassifierModel.fit(X_train, y_train)

#Calculating Details
print('RandomForestClassifierModel Train Score is : ' , RandomForestClassifierModel.score(X_train, y_train))
print('RandomForestClassifierModel Test Score is : ' , RandomForestClassifierModel.score(X_test, y_test))
print('RandomForestClassifierModel features importances are : ' , RandomForestClassifierModel.feature_importances_)
print('----------------------------------------------------')

#Calculating Prediction
y_pred = RandomForestClassifierModel.predict(X_test)
y_pred_prob = RandomForestClassifierModel.predict_proba(X_test)
print('Predicted Value for RandomForestClassifierModel is : ' , y_pred[:10])
print('Prediction Probabilities Value for RandomForestClassifierModel is : ' , y_pred_prob[:10])

In [ ]:
#calculating f1 score

F1Score = f1_score(y_test, y_pred, average='micro') #it can be : binary,macro,weighted,samples
print('F1 Score is : ', F1Score)

In [ ]:
#Import Libraries
from sklearn.metrics import classification_report
#----------------------------------------------------

#----------------------------------------------------
#Calculating classification Report :  
#classification_report(y_true, y_pred, labels=None, target_names=None,sample_weight=None, digits=2, output_dict=False)

ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : ', ClassificationReport )

In [ ]:
#Calculating Confusion Matrix
CM = confusion_matrix(y_test, y_pred)
print('Confusion Matrix is : \n', CM)

# drawing confusion matrix
sns.heatmap(CM, center = True,annot=True,fmt='.1f')
plt.show()

# Handling our test data to predict it

In [ ]:
#imprting our test data 
test=pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_test.csv')

In [ ]:
#some rows from our test data
test.head()

In [ ]:
test.info()

In [ ]:
# number of nulls 
test.isnull().sum()

We shouldn't drop any rows from our test dataset because they are our finally target ,so we will handel our missing values

First we will drop company size and company type as we droped them from our training data

In [ ]:
# dropping columns which we removed when training our model
test.drop(['company_size','company_type','enrollee_id','city'],axis=1,inplace=True)
test.head()

In [ ]:
test['gender']=test['gender'].replace({'Male':0,'Female':1,'Other':2})
test['enrolled_university']=test['enrolled_university'].replace({'no_enrollment':0,'Full time course':1,'Part time course':2})
test['relevent_experience']=test['relevent_experience'].replace({'No relevent experience':0,'Has relevent experience':1})
test['education_level']=test['education_level'].replace({'Graduate':1,'High School':4,'Primary School':6,'Masters':2,'Phd':3})

test['major_discipline']=test['major_discipline'].replace({'STEM':1, 'Humanities':2, 'Arts':3, 'Business Degree':4,'No Major':5,'Other':6})

test['experience']=test['experience'].replace({'>20':30,'<1':0})

test['last_new_job']=test['last_new_job'].replace({'>4':5,'1':1,'never':0,'4':4, '3':3, '2':2})
test['experience']=test['experience'].replace({'>20':30,'<1':0,'15':15,'13':13,'7':7, '5':5, '16':16, '4':4, '11':11, '18':18, '19':19, '12':12,
       '10':10, '9':9, '2':2, '6':6, '14':14, '3':3, '8':8, '20':20, '17':17, '1':1})


We will fill nulls with most frequent value in each column

In [ ]:
test['gender'].fillna(test['gender'].mode()[0],inplace=True)
test['enrolled_university'].fillna(test['enrolled_university'].mode()[0],inplace=True)
test['education_level'].fillna(test['education_level'].mode()[0],inplace=True)
test['major_discipline'].fillna(test['major_discipline'].mode()[0],inplace=True)
test['last_new_job'].fillna(test['last_new_job'].mode()[0],inplace=True)
test['experience'].fillna(test['experience'].mean(),inplace=True)


In [ ]:
test.info()

In [ ]:
test.head()

# Predicting our test data 

In [ ]:
test_pred= RandomForestClassifierModel.predict(test)
test_pred

If you find this kernal useful for you please upvote , to encourage me to do more